In [2]:
import os
import pandas as pd
import requests
import time

def load_table(table_name, batch_size=1000, pause=0.5, max_batch=None):
    base_url = f'https://data.epa.gov/efservice/{table_name}/{{start}}:{{end}}/JSON'
    todos_los_datos = []
    start = 1
    lote = 0

    while True:
        end = start + batch_size - 1
        url = base_url.format(start=start, end=end)
        print(f"Descargando: {table_name} [{start}:{end}]")

        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            data = response.json()
        except Exception as e:
            print(f" Error en el request: {e}")
            break

        if not data:
            print(" Fin de los datos.")
            break

        todos_los_datos.extend(data)
        start += batch_size
        lote += 1

        if max_batch and lote >= max_batch:
            print(f" Límite de lotes alcanzado ({max_batch})")
            break

        time.sleep(pause)

    return pd.DataFrame(todos_los_datos)

def cargar_o_actualizar_tabla(nombre_tabla, carpeta='epa_cache', forzar_actualizacion=False):
    os.makedirs(carpeta, exist_ok=True)
    ruta_archivo = os.path.join(carpeta, f"{nombre_tabla}.csv")

    if os.path.exists(ruta_archivo) and not forzar_actualizacion:
        print(f"📁 Leyendo desde cache: {ruta_archivo}")
        return pd.read_csv(ruta_archivo)

    print(f"🔄 Descargando datos desde la API para: {nombre_tabla}")
    df = load_table(nombre_tabla)

    print(f"💾 Guardando cache local en: {ruta_archivo}")
    df.to_csv(ruta_archivo, index=False)

    return df

# Cargar tabla de instalaciones
df_facilities = cargar_o_actualizar_tabla("ghg.rlps_ghg_emitter_facilities")

# Cargar tabla de emisiones por gas
df_emissions = cargar_o_actualizar_tabla("ghg.EF_W_EMISSIONS_SOURCE_GHG")


📁 Leyendo desde cache: epa_cache\ghg.rlps_ghg_emitter_facilities.csv
📁 Leyendo desde cache: epa_cache\ghg.EF_W_EMISSIONS_SOURCE_GHG.csv


In [ ]:
df_emissions = cargar_o_actualizar_tabla("ghg.EF_W_EMISSIONS_SOURCE_GHG", forzar_actualizacion=True)
df_facilities = cargar_o_actualizar_tabla("ghg.rlps_ghg_emitter_facilities", forzar_actualizacion=True)

In [8]:
url = 'https://data.epa.gov/efservice/ghg.rlps_ghg_emitter_facilities/1:100/JSON'
df_emissions = pd.read_json(url)

df_emissions.head()

,add_naics_code,address1,address2,cems_used,city,cogen_unit_emm_ind,county,county_fips,epa_verified,facility_id,...,latitude,longitude,parent_company,plant_code,primary_naics,secondary_naics,state,state_name,year,zip
0,NaN,5105 LAKE TERRELL ROAD,NaN,None,FERNDALE,Y,WHATCOM COUNTY,53073,None,1000001,...,48.828707,-122.685533,"EMPECO IV, LLC AND USPF II FERNDALE HOLDINGS, ...",None,221112,NaN,WA,WASHINGTON,2010,98248
1,NaN,5105 LAKE TERRELL ROAD,NaN,None,FERNDALE,Y,WHATCOM COUNTY,53073,None,1000001,...,48.828707,-122.685533,"EMPECO IV, LLC AND USPF II FERNDALE HOLDINGS, ...",None,221112,NaN,WA,WASHINGTON,2011,98248
2,NaN,5105 LAKE TERRELL ROAD,NaN,None,FERNDALE,N,WHATCOM COUNTY,53073,None,1000001,...,48.828707,-122.685533,PUGET SOUND ENERGY (100%),None,221112,NaN,WA,WASHINGTON,2012,98248
3,NaN,5105 LAKE TERRELL ROAD,NaN,None,FERNDALE,N,WHATCOM COUNTY,53073,Y,1000001,...,48.828707,-122.685533,PUGET SOUND ENERGY (100%),Y,221112,NaN,WA,WASHINGTON,2013,98248
4,NaN,5105 LAKE TERRELL ROAD,NaN,None,FERNDALE,N,WHATCOM COUNTY,53073,None,1000001,...,48.828707,-122.685533,PUGET SOUND ENERGY (100%),Y,221112,NaN,WA,WASHINGTON,2014,98248


In [7]:
url = 'https://data.epa.gov/efservice/ghg.EF_W_EMISSIONS_SOURCE_GHG/1:100/JSON'
df_emissions = pd.read_json(url)

df_emissions.head(50)

,bamm_desc_source_summary,bamm_indicator_source_summary,basin_associated_with_facility,facility_id,facility_name,industry_segment,reporting_category,reporting_year,total_reported_ch4_emissions,total_reported_co2_emissions,total_reported_n2o_emissions
0,None,None,None,1003447,UGC - DRAGON TRAIL GAS PLANT - 08-103-00036,Onshore natural gas processing [98.230(a)(3)],Enhanced Oil Recovery Hydrocarbon Liquids [98....,2016,NaN,0.0,NaN
1,None,None,None,1004777,Great Lakes Gas Transmission Pipeline Shevlin ...,Onshore natural gas transmission compression [...,Completions and Workovers with Hydraulic Fract...,2023,0.000000,0.0,0.000000
2,None,None,"220 - Gulf Coast Basin (LA, TX)",1009706,"SilverBow Resources, LLC - Basin 220",Onshore petroleum and natural gas production [...,Completions and Workovers without Hydraulic Fr...,2020,0.000000,0.0,0.000000
3,None,None,None,1012154,580 - San Juan Basin Gathering/Boosting,Onshore petroleum and natural gas gathering an...,Dehydrators [98.236(e)],2018,3770.830000,1223.8,0.000000
4,None,None,None,1004849,CONROE NATURAL GAS PROCESSING PLANT,Onshore natural gas processing [98.230(a)(3)],Reciprocating Compressors [98.236(p)],2015,73.020000,4.4,NaN
5,None,No,,1009659,"Enervest Operating, L.L.C. 260 East Texas Basin",Onshore petroleum and natural gas production [...,AssociatedGasVentingFlaringDetails,2014,0.000000,0.0,0.000000
6,None,None,None,1011274,Kensington Cryogenic Processing Plant,Onshore natural gas processing [98.230(a)(3)],Acid Gas Removal Units [98.236(d)],2022,NaN,3594.8,NaN
7,None,None,540 - Denver Basin,1009354,HighPoint Operating Corporation - Denver Basin...,Onshore petroleum and natural gas production [...,Equipment Leaks Surveys and Population Counts ...,2015,18.750000,1.7,NaN
8,None,None,None,1004400,Springridge South Compressor Station,Onshore natural gas processing [98.230(a)(3)],Centrifugal Compressors [98.236(o)],2023,0.000000,0.0,NaN
9,None,None,None,1003535,Eastern Gas Transmission and Storage - Hasting...,Onshore natural gas transmission compression [...,Associated Gas Venting and Flaring [98.236(m)],2018,0.000000,0.0,0.000000


In [ ]:
df = pd.merge(df_facilities, df_emissions, on='facility_id', how='left')

df.head(10)

In [ ]:
import plotly.graph_objects as go

df_emissions = df_emissions[['reporting_year', 'industry_segment', 'total_reported_ch4_emissions']].copy()
df_emissions.dropna(subset=['reporting_year', 'industry_segment', 'total_reported_ch4_emissions'], inplace=True)

# Agrupamos por año y segmento
grouped = df_emissions.groupby(['reporting_year', 'industry_segment'])['total_reported_ch4_emissions'].sum().reset_index()

# Pivot para tener columnas por industry_segment (para graficar apilado)
pivoted = grouped.pivot(index='reporting_year', columns='industry_segment', values='total_reported_ch4_emissions')
pivoted.fillna(0, inplace=True)

fig = go.Figure()

for segment in pivoted.columns:
    fig.add_trace(
        go.Bar(
            x=pivoted.index,
            y=pivoted[segment],
            name=segment
        )
    )

fig.update_layout(
    barmode='stack',
    title='Methane Emissions vs Year (Stacked by Industry Segment)',
    xaxis_title='Year',
    yaxis_title='Methane Emissions (tons)',
    legend_title='Industry Segment',
    height=600
)

fig.show()

In [ ]:
df_emissions.groupby('reporting_year')['total_reported_ch4_emissions'].sum().plot(kind='bar', title='Total CH4 Emissions per Year')


In [ ]:
df_emissions[df_emissions['reporting_year'] == 2019].groupby('industry_segment')['total_reported_ch4_emissions'].sum().sort_values(ascending=False).head(10)
